In [1]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import os, sys
cwd = os.getcwd()
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [2]:
#parent_dir = os.path.abspath(os.path.join(cwd, os.pardir, 'SBMLLint'))
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
print(parent_dir)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [3]:
os.chdir(parent_dir)
print(os.getcwd())

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [4]:
#import os, sys, inspect
#import inspect
#cmd_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0]))
#if cmd_folder not in sys.path:
#     sys.path.insert(0, cmd_folder)

In [5]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.games import print_model as pm

In [6]:
os.getcwd()

'/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint'

In [7]:
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [8]:
num = 35
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
pm.print_model(model)

<Model vilarOscillator "Vilar2002_Oscillator">
Reaction1: A + R -> C;
Reaction2: A -> EmptySet;
Reaction3: C -> R;
Reaction4: R -> EmptySet;
Reaction5: A + DA -> DAp;
Reaction6: DAp -> A + DA;
Reaction7: DA -> DA + MA;
Reaction8: DAp -> DAp + MA;
Reaction9: MA -> EmptySet;
Reaction10: MA -> A + MA;
Reaction11: A + DR -> DRp;
Reaction12: DRp -> A + DR;
Reaction13: DR -> DR + MR;
Reaction14: DRp -> DRp + MR;
Reaction15: MR -> EmptySet;
Reaction16: MR -> MR + R;


In [9]:
print(Molecule)
print(Reaction)

<class 'SBMLLint.common.molecule.Molecule'>
<class 'SBMLLint.common.reaction.Reaction'>


In [10]:
reaction1 = model.getReaction(0)
reaction2 = model.getReaction(2)

In [11]:
Reaction(reaction1)
Reaction(reaction2)
Reaction.reactions

[A + R -> C, C -> R]

In [12]:
reaction1.getReactant(0).species

'A'

In [13]:
Molecule.molecules

[A, R, C]

In [14]:
m = Molecule.getMolecule('C')
if m is None:
    print('hah')
else: 
    print(m)

C


In [15]:
print(Reaction.reactions[1].reactants)
print(Reaction.reactions[1].products)

[MoleculeStoichiometry(molecule=C, stoichiometry=1.0)]
[MoleculeStoichiometry(molecule=R, stoichiometry=1.0)]


In [16]:
#ReactionCategory = collections.namedtuple('ReactionCategory',
#    'category predicate')
#MoleculeStoichiometry = collections.namedtuple('Molecule', 'molecule stoichiometry')
#MoleculeStoichiometry(molecule = Molecule.molecules[0], stoichiometry = 2)

In [17]:
"""Set of Molecules(SOM)"""

from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML

import numpy as np

In [18]:
BRACKET_OPEN = "{"
BRACKET_CLOSE = "}"

class SOM(object):
    soms = []  # All SOMs. 
    def __init__(self, molecule):
        """
        :param molecules Molecule instances:
        :param reactions Reaction instances:
        """
        self.molecules = {molecule}
        self.reactions = set()
        self.identifier = self.makeId()
        self.__class__.addSOM(self)

    def __repr__(self):
        return self.identifier        
        
    def makeId(self):
        """
        Creates an identifier for the SOM to uniquely
        identifies its elements.
        :return str:
        """
        def joinMoleculeNames(molecules):
          names = [m.name for m in molecules]
          names.sort()
          return ', '.join(names)
        #
        identifier = "%s%s%s" % (
            BRACKET_OPEN, 
            joinMoleculeNames(list(self.molecules)),
            BRACKET_CLOSE
            )
        return identifier
        
    @classmethod    
    def addSOM(cls, new_som):
        if any([new_som.molecules.intersection(s.molecules) for s in cls.soms]):
          pass
        else:
          cls.soms.append(new_som)
    
    @classmethod
    def findSOM(cls, molecule):
        """
        Find the SOM that contains molecule
        and returns SOM
        param Molecule instance molecule
        """    
        for som in cls.soms:
            #if molecule in som.molecules:
            #    return som
            for m in som.molecules:
                if molecule.name == m.name:
                    return som
            
    @classmethod
    def merge(cls, reaction):
        """
        Merges two SOMs using a UniUni reaction 
        and updates cls.soms
        : return merged SOM
        """ 
        som1 = cls.findSOM(reaction.reactants[0].molecule)
        som2 = cls.findSOM(reaction.products[0].molecule)

        som1.molecules = som1.molecules.union(som2.molecules)
        som1.reactions.add(reaction)
        som1.identifier = som1.makeId()
        cls.soms.remove(som2)
        return som1

In [19]:
Molecule.molecules

[A, R, C]

In [20]:
SOM.soms

[]

In [22]:
rct1 = Reaction.reactions[1].reactants[0].molecule
print(rct1)

pdt1 = Reaction.reactions[0].products[0].molecule
print(pdt1)

C
C


In [23]:
SOM(Molecule.molecules[0])
SOM(Molecule.molecules[1])
SOM(Molecule.molecules[2])
SOM(pdt1)
SOM(rct1)
print(SOM.soms)

[{A}, {R}, {C}]


In [24]:
Reaction.reactions

[A + R -> C, C -> R]

In [30]:
print(SOM.findSOM(rct1))
print(SOM.findSOM(pdt1))

{C}
{C}


In [32]:
SOM.soms

[{A}, {R}, {C}]

In [31]:
Reaction.reactions[1]

C -> R

In [33]:
SOM.merge(Reaction.reactions[1])

{C, R}

In [34]:
print(SOM.soms)

[{A}, {C, R}]


In [133]:
    def reduce(self, som):
        rct_subset = set(self.reactants).intersection(som)
        pdt_subset = set(self.products).intersection(som)
        rct_index = 0
        pdt_index = 0
        
        # Return False if the reaction is not reducible 
        if (not bool(rct_subset)) | (not bool(pdt_subset)):
            return False
        
        while bool(rct_subset) & bool(pdt_subset):
            rct = self.reactants[rct_index]
            pdt = self.products[pdt_index]

            # find reactant and product within given som
            while (rct not in rct_subset):
                rct_index = rct_index + 1
                rct = self.reactants[rct_index]
            while (pdt not in pdt_subset):
                pdt_index = pdt_index + 1
                pdt = self.products[pdt_index]

            # subtract stoichiometry from each side
            self.stoichiometry[0][rct_index] = self.stoichiometry[0][rct_index] - 1
            self.stoichiometry[1][pdt_index] = self.stoichiometry[1][pdt_index] - 1

            # remove element if stoichiometry becomes zero
            if self.stoichiometry[0][rct_index] == 0:
                rct_subset.remove(rct)
                del self.reactants[rct_index]
                del self.stoichiometry[0][rct_index]

            if self.stoichiometry[1][pdt_index] == 0:
                pdt_subset.remove(pdt)
                del self.products[pdt_index]
                del self.stoichiometry[1][pdt_index]
        
        # Update category because the reaction was reduced
        self.updateCategory()
        
        return True
  
    def updateCategory(self):
        
        # if either side is empty, we set None for the category and stoichiometry
        if (len(self.reactants)==0) | (len(self.products)==0) | \
           (self.reactants==[cn.EMPTYSET]) | (self.products==[cn.EMPTYSET]):
            self.category = None
        elif (len(self.reactants)==1) & (len(self.products)==1):
            self.category = cn.UNIUNI
        elif (len(self.reactants)==1) & (len(self.products)!=1):
            self.category = cn.UNIMULTI
        elif (len(self.reactants)!=1) & (len(self.products)==1):
            self.category = cn.MULTIUNI
        else:
            self.category = cn.MULTIMULTI
        print("Checking category...")

In [39]:
Reaction.reactions[0]._getCategory()

'reaction_n_1'